In [1]:
import pickle as pk
import pandas as pd
import numpy as np
import datetime
import matplotlib.dates as dates

In [2]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_F.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptF = pd.DataFrame(botpt_data)
df_botptF['bottom_pressure'] = df_botptF['bottom_pressure'].astype(float)
df_botptF['depth']=df_botptF['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptF.index.to_pydatetime()]
df_botptF['epoch'] = epoch
df_botptF= df_botptF.sort_index()
df_botptF.index.name= 'Date'
del df_botptF['epoch']
del df_botptF['bottom_pressure']
df_botptF.tail()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/data/botpt/bottom_pressure15S_F.pkl'

In [3]:
# df_botptF.loc['2018-02-17']

In [ ]:
with open('/home/jovyan/data/botpt/bottom_pressure15S_E.pkl', 'rb') as E:
    botpt_data = pk.load(E)
df_botptE = pd.DataFrame(botpt_data)
df_botptE['bottom_pressure'] = df_botptE['bottom_pressure'].astype(float)
df_botptE['depth']=df_botptE['bottom_pressure'].astype(float) * 0.670
#MJ03F_cal_depths = [MJ03F_pressure * 0.0670 for MJ03F_pressure in MJ03F_pressure]
#list comprehention
epoch= [i.timestamp() for i in df_botptE.index.to_pydatetime()]
df_botptE['epoch'] = epoch
df_botptE= df_botptE.sort_index()
df_botptE.index.name= 'Date'
del df_botptE['epoch']
del df_botptE['bottom_pressure']
df_botptE.head()

#### Merge BOTPT E and BOTPT F 

In [ ]:
test = pd.merge(df_botptF, df_botptE,how='outer', indicator=True, left_index=True, right_index=True, suffixes=('_F', '_E'))

In [ ]:
df_botptMerge = test[test['_merge'] == 'both']
del df_botptMerge['_merge']
del df_botptF
del df_botptE

In [ ]:
df_botptMerge = df_botptMerge.loc['2017-1-1 00:00:00':'2017-01-30 00:00:00']

In [ ]:
df_botptMerge

#### Calculate Depth difference 

In [ ]:
depthDiff = df_botptMerge['depth_E'].values - df_botptMerge['depth_F'].values

In [ ]:
depthDiff

In [ ]:
# depthDiff = df_botptMerge['diff'].abs()
# depthDiff = df_botptMerge['diff'].abs()
# df_botptMerge['diff'] = depthDiff.abs()

In [ ]:
df_botptMerge['diff'] = depthDiff
# df_botptMerge['diff'] = abs(depthDiff)

In [ ]:
df_botptMerge['diff'].head(5)

In [ ]:
depthDiff = df_botptMerge['diff'].abs()

In [ ]:
depthDiff.head()

In [ ]:
df_botptMerge['diff'] = depthDiff

#### Create time and height vectors for plotting 

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_botptMerge['diff'].tolist()
time_int = []
time = list(pd.to_datetime(df_botptMerge.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

#### Use Groupby to create one day mean measurements

In [ ]:
df_botptMerge['date']=pd.DatetimeIndex(df_botptMerge.index).date
df_botptMerge

In [ ]:
df_botptMean=df_botptMerge.groupby('date').mean()
df_botptMean.tail(100)

In [ ]:
df_test = df_botptMean.head(1000)
df_test.head(10)

In [ ]:
df_test['newdiff'] = df_test['diff'].diff()
df_test.head(10)

In [ ]:
max = 0
for index,row in df_test.iterrows():
    if row['diff']>max:
        max = row['diff']
        df_test.at[index,'state'] = 2
    else:
        if row['newdiff']>0:
            df_test.at[index,'state'] = 1
        else:
            df_test.at[index,'state'] = -1
    df_test.at[index,'new'] = max
        
df_test.head(20)

In [ ]:
df_test.state.plot(marker='.',linestyle='')

In [ ]:
# q= df_botptMean['diff'].values
# q[0:3]
# pd.DataFrame({"A": [1, 2], "B": [3, 4]}).to_numpy()
# test = pd.DataFrame(df_botptMean['diff']).to_numpy()

# for i in range(len(test)):
#     if i==0:
#         #diff_max=test[0]
#         print(0)
#     else:
#         diff_max = max(test[:i])
#         print(test[i]-diff_max)

In [ ]:
# test = pd.DataFrame(df_botptMean['diff']).to_numpy()

# d={}
# for i in range(len(test)):
#     if i==0:
#         #diff_max=test[0]
#         print(0)
#     else:
#         diff_max = max(test[:i])
#         print(test[i]-diff_max)

In [ ]:
#DO NOT CHANGE
# test = pd.DataFrame(df_botptMean['diff']).to_numpy()

# for i in range(len(test)):
#     if i==0:
#         #diff_max=test[0]
#         print(0)
#     else:
#         diff_max = max(test[:i])
#         daily_diff = (test[i]-diff_max)
#         print(daily_diff)

In [ ]:
test = pd.DataFrame(df_botptMean['diff']).to_numpy()
# type(test)

for i in range(len(test)):
    if i==0:
        #diff_max=test[0]
        print(0)
    else:
        diff_max = max(test[:i])
        daily_diff = (test[i]-diff_max)
        print(daily_diff)

In [ ]:
# use list to save results
test = pd.DataFrame(df_botptMean['diff']).to_numpy()

results = []
for i in range(len(test)):
    if i==0:
        #diff_max=test[0]
        print(0)
    else:
        diff_max = max(test[:i])
        daily_diff = (test[i]-diff_max)
    results.append(daily_diff(100))

# get output array: 
# np.hstack(results)

In [ ]:
len(daily_diff)

In [ ]:
df_diff = pd.DataFrame(daily_diff)

In [ ]:
df_diff.head()

In [ ]:
df_botptMean['diff'].plot()
# plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)

In [ ]:
df_botptMean['daily_diff'] = df_botptMean['diff'] 

In [ ]:
import pickle


with open('/jovyan/data/botpt/2019bottom_pressure15s_E.pkl, 'rb') as f:
    botpt_E = pickle.load(f)

In [ ]:
botpt_E

In [ ]:
with open('2019bottom_pressure15s_F.pkl', 'rb') as f:
    botpt_F = pickle.load(f)

In [ ]:
botpt_F.iloc[1, 10]

In [ ]:
botpt_F[1, 10]